# Real example: finding near-duplicate headlines in a collection of news stories

Here is a real case I've run into in my work at Brandwatch where parallelising the work on a cluster would have speeded up my progress. (It might be a slightly odd example but at least it's a real one!)

In [1]:
import pandas as pd
from difflib import SequenceMatcher
from dask.distributed import Client

pd.set_option('display.max_colwidth', -1)

Here is a sample dataset of news headlines from 9 stories, with 1000 headlines from each story.

In [2]:
df = pd.read_csv("sample_news_headlines.csv")

In [3]:
df.head()

,title,story
0,Sensational: Hungarian Junior Handball Team triumphs at World Championship! | Daily News Hungary,france-football
1,World Cup afterglow gives France a sorely needed boost,france-football
2,"World Cup final 2018: The sad story of Ivan Perisic, today's decisive player but not always in the way he might want",france-football
3,LIVE| FIFA World Cup 2018 final: History awaits Croatia in summit clash vs France,france-football
4,VAR used for first time in World Cup final,france-football


In [4]:
df['story'].value_counts().to_frame(name='Frequency')

,Frequency
stephen-hawking,1000
brexit-resignations,1000
nhs-cyber-attack,1000
may-deal,1000
france-football,1000
macron,1000
nhs-70-years,1000
brexit-trump,1000
gatwick-drone,1000


Here is some code, self-plagiarised from a previous workshop, for identifying and excluding near-duplicates.

In [5]:
# Here is one way to define when two strings are near-duplicates, using their longest common substring (LCS).

def is_near_duplicate(str1, str2):

    lcs_fraction_threshold = 0.50
    
    min_length_threshold = 40   # For very short strings things get a bit weird, so we have a threshold on that too

    str1 = str1.casefold()
    str2 = str2.casefold()
    
    len1 = len(str1)
    len2 = len(str2)
    
    # Be careful with SequenceMatcher; if you don't pass autojunk=False it will use its "autojunk heuristic"
    # (which you can read about online) and you won't get all the matches you expect.
    
    match = SequenceMatcher(None, str1, str2, autojunk=False).find_longest_match(0, len1, 0, len2)

    if (    (match.size > lcs_fraction_threshold * len1 and len1 >= min_length_threshold) 
             or  (match.size > lcs_fraction_threshold * len2 and len2 >= min_length_threshold)):
        return True, str1[match.a: (match.a + match.size)]
    else:
        return False, None


In [6]:
# This function excludes near-duplicate mentions from a dataframe, as defined above by is_near_duplicate

def exclude_near_dupes(df, text_col_name):

    df = df.copy()
    num_rows = df.shape[0]

    # Create new columns with indices to match the dataframe.
    accepted       = df[text_col_name].apply(lambda x: False)
    near_dupe_pos  = df[text_col_name].apply(lambda x: -1)
    near_dupe_text = df[text_col_name].apply(lambda x: None)
    common_text    = df[text_col_name].apply(lambda x: None)

    for i in range(num_rows):

        acc = True

        for j in range(i):

            b, s = is_near_duplicate(df[text_col_name].iloc[i], df[text_col_name].iloc[j])

            if b and accepted.iloc[j]:
                acc = False
                pos = j
                dupe_txt = df[text_col_name].iloc[j]
                common_txt = s

                break

        if acc:
            accepted.iloc[i] = True

        else:
            near_dupe_pos.iloc[i]  = pos
            near_dupe_text.iloc[i] = dupe_txt
            common_text.iloc[i]    = common_txt


    df['NearDupePos']  = near_dupe_pos
    df['NearDupeText'] = near_dupe_text
    df['CommonText']   = common_text

    accepted_df = df.loc[accepted].copy().drop(columns=['NearDupePos', 'NearDupeText', 'CommonText'])
    rejected_df = df.loc[~ accepted].copy()

    return accepted_df, rejected_df


The string matching operations used to detect near-duplicates take quite a bit of CPU, and the cost grows quadratically with the number of rows (because we're testing each pair of rows).

Here we run it just on the headlines from the France football story:

In [7]:
%%time
accepted_df, rejected_df = exclude_near_dupes(df=df.loc[df['story'] == "france-football"], text_col_name='title')

CPU times: user 40.8 s, sys: 62.6 ms, total: 40.9 s
Wall time: 40.9 s


Thus if we work through each news story in turn, in a sequential way, we will end up waiting a little while:

In [8]:
def run_locally():
    
    story_dfs = list(map(lambda story: df.loc[df['story'] == story], df['story'].unique()))
    results = list(map(lambda story_df: exclude_near_dupes(story_df, text_col_name='title'), story_dfs))

    accepted_dfs = list(map(lambda x: x[0], results))
    rejected_dfs = list(map(lambda x: x[1], results))
    
    return pd.concat(accepted_dfs, ignore_index=True), pd.concat(rejected_dfs, ignore_index=True)


In [9]:
%%time
local_accepted_df, local_rejected_df = run_locally()

CPU times: user 6min 26s, sys: 5.84 s, total: 6min 32s
Wall time: 6min 33s


Now let's process the new stories in parallel, using our cluster:

In [8]:
client = Client('127.0.0.1:8786')

In [9]:
client

Client Scheduler: tcp://127.0.0.1:8786 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 48.00 GB


The way I've written the code above, the only changes needed to make it run distributed instead are:

1. **We used `client.map` instead of a regular `map`**
2. **We needed an extra called to `client.gather` to gather in the results**


In [10]:
def run_distributed():
    
    story_dfs = list(map(lambda story: df.loc[df['story'] == story], df['story'].unique()))

    results_futures = client.map(lambda story_df: exclude_near_dupes(story_df, text_col_name='title'), story_dfs)
    results = client.gather(results_futures)

    accepted_dfs = list(map(lambda x: x[0], results))
    rejected_dfs = list(map(lambda x: x[1], results))
    
    return pd.concat(accepted_dfs, ignore_index=True), pd.concat(rejected_dfs, ignore_index=True)


In [11]:
%%time
distributed_accepted_df, distributed_rejected_df = run_distributed()

CPU times: user 148 ms, sys: 20.1 ms, total: 169 ms
Wall time: 1min 5s


In [15]:
distributed_accepted_df.equals(local_accepted_df)

True

In [16]:
distributed_rejected_df.equals(local_rejected_df)

True

This problem was well-suited to a solution with `client.map` because:

 - The work was **easy to break up** into chunks of equal sizes (in fact, it was basically already broken up that way, because of the different news stories)
 - For each news story **the data was fairly small** - meaning that serialising/pickling it and sending it over the network was quick - **but the CPU required was high**

Another problem like this would be doing a grid search of hyperparameters for a model fitting and evaluation task, where the training and testing data is moderately sized.

For some types of problems, the work does not break down into chunks as neatly as this, in which case it becomes tedious for us as data scientists to have to program the division of work into subtasks. At this point we can start to use dask's **higher level APIs such as distributed dataframes**, which are similar to Spark's but **aim to behave as much like pandas dataframes as possible**.